In [1]:
# Import required libraries
import pandas as pd
import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
import plotly.express as px
from jupyter_dash import JupyterDash
import plotly.offline as pyo
pyo.init_notebook_mode(connected=True)
import plotly.graph_objects as go
import shutup
shutup.please()

In [2]:
# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()
spacex_df['Mission Outcome'] = [x[:7] for x in spacex_df['Mission Outcome']]
spacex_df.drop('Unnamed: 0', axis=1, inplace=True)

In [3]:
# Create a dash application
app = JupyterDash(__name__)
#suppress_callback_exceptions=True)

In [4]:
# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                # dcc.Dropdown(id='site-dropdown',...)
                                dcc.Dropdown(id='site-dropdown',
                                            options=[{'label': 'All Sites', 'value': 'ALL'},
                                                     {'label': 'CCAFS LC-40 ', 'value': 'CCAFS LC-40'},
                                                     {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
                                                     {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},
                                                     {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'}],
                                            value='ALL',
                                            placeholder='Select a Launch Site here',
                                            searchable=True),
                                html.Br(),
                                # pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div([ ],id='success-pie-chart'),
                                html.Br(),
                                html.P("Payload range (Kg):"),
                                # slider to select payload range
                                #dcc.RangeSlider(id='payload-slider',...)
                                dcc.RangeSlider(id='payload-slider',
                                                min=0, max=10000, step=1000,
                                                marks={0: '0',100: '100'},
                                                value=[min_payload, max_payload]),
                                # scatter chart to show the correlation between payload and launch success
                                html.Br(),
                                html.Div([ ],id='success-payload-scatter-chart'),
                                ])

In [5]:
# callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback([Output(component_id='success-pie-chart', component_property='children')],
              [Input(component_id='site-dropdown', component_property='value')])
def update_pie(site):
    if site != 'ALL':
        df =  spacex_df[spacex_df['Launch Site']==site]
        pie_fig = px.pie(df, values='class', names='Launch Site', title=f'Total Success Launches By {site}')
        return [dcc.Graph(figure=pie_fig)]
    else:
        pie_fig = px.pie(spacex_df, values='class', names='Launch Site', title=f'Total Success Launches By {site}')
        return [dcc.Graph(figure=pie_fig)]

In [6]:
# callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback([Output(component_id='success-payload-scatter-chart', component_property='children')],
              [Input(component_id='site-dropdown', component_property='value'), Input(component_id="payload-slider", component_property="value")])
def update_scatter(site,slider):
    if site != 'ALL':
        df =  spacex_df[spacex_df['Launch Site']==site]
        df = df[(df['Payload Mass (kg)'] >= slider[0]) & (df['Payload Mass (kg)'] <= slider[1])]
        scatter_fig= px.scatter(df, x='Payload Mass (kg)', y='class', color='Booster Version Category', title=f'Payload VS.Success Launches By {site}')
        return [dcc.Graph(figure=scatter_fig)]
    else:
        df = spacex_df[(spacex_df['Payload Mass (kg)'] >= slider[0]) & (spacex_df['Payload Mass (kg)'] <= slider[1])]
        scatter_fig= px.scatter(df, x='Payload Mass (kg)', y='class', color='Booster Version Category', title=f'Payload VS.Success Launches By {site}')
        return [dcc.Graph(figure=scatter_fig)]

In [7]:
# Run the app
if __name__ == '__main__':
    app.run_server(mode="inline",port=8501);